In [9]:
import sys
import os
import matplotlib.pyplot as plt
from vis.deprecated_vis import imshow
import data.PATHS
from vis.plot2D import * 


# Prepare Data

In [10]:

from loss.flow import GeneralLoss
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from vis.deprecated_vis import imshow, visualize_points3D, visualize_multiple_pcls
from data.dataloader import SFDataset4D
import torch
from pytorch3d.ops.knn import knn_points
from torch_scatter import scatter
from sklearn.cluster import DBSCAN
from pytorch3d.transforms import axis_angle_to_matrix

dataset = SFDataset4D(dataset_type='argoverse', n_frames=4)
data = dataset[0]
device = torch.device('cuda:0')
pc1 = data['pc1'].to(device)
pc2 = data['pc2'].to(device)
gt_flow = data['gt_flow'].to(device)

# reduce radius for computation reasons
pc = pc1
max_radius = 10
for i in range(len(pc)):
    mask = torch.norm(pc[i], dim=-1) < max_radius   
    gt_flow = gt_flow[:, mask]    
    pc = pc[:, mask]
    
pc = torch.stack([torch.cat((pc[i], i * torch.ones_like(pc[i][:, :1])), dim=-1) for i in range(len(pc))])

SmoothLosses = [GeneralLoss(pc1=pc[i:i+1, :, :3], pc2=pc[i:i+1, :, :3], K=16, smooth_weight=1) for i in range(len(pc1) - 1)]
# visualize_multiple_pcls(*[pc[i].detach().cpu().numpy() for i in range(len(pc))])

db_clusters = DBSCAN(eps=0.2, min_samples=5).fit_predict(pc[0].detach().cpu().numpy())
db_clusters = torch.tensor(db_clusters).to(device)

M = len(torch.unique(db_clusters))
logs_mask = torch.rand(pc.shape[0], pc.shape[1], M, device=device, requires_grad=True)

IndexError: list index out of range

# Match Instances

In [ ]:
optimizer = torch.optim.Adam([logs_mask], lr=0.01)



# todo box visuals, overlap, background without box
# [x] todo transform clusters to probability masks
# todo inter-sequence instance geometrical registration for same predictions

v_min = scatter(pc[0, :, :3], db_clusters + 1, dim=0, reduce='min')
v_max = scatter(pc[0, :, :3], db_clusters + 1, dim=0, reduce='max')

centers = (v_min + v_max) / 2
sizes = (v_max - v_min)

# draw boxes
boxes = torch.cat((centers, sizes), dim=-1)

# just rotate and get points inside

vis_boxes = torch.cat((boxes, torch.zeros_like(boxes[:, :1], device=boxes.device)), dim=-1) # yaw angle
yaw = vis_boxes[:, -1]

rotations = torch.cat((torch.zeros(len(boxes), 2, device=device), yaw[:, None]), dim=1)

rot_mats = axis_angle_to_matrix(rotations)
# rot_mats = rot_mats.permute(0,2,1) # should be there?

out = pc[:1, :, :3] - boxes[:, None, :3]
out = torch.bmm(out, rot_mats) #+ boxes[:, None, :3]    # shifted to box origin and rotated

logic_mask = (out <= boxes[:, None, 3:6] / 2) & (out >= - boxes[:, None, 3:6] / 2)  
inside_mask = logic_mask.all(dim=-1).T # all dimensions must be inside box ---> boolean mask of clusters

# todo overlap where multiple overlapping boxes decrease the probability
box_probs = (db_clusters[:, None] * inside_mask)

# split to cases?
for i in range(200):
    mask_without_background = logs_mask.softmax(dim=-1)[:1,:,1:]
    
    overlaps = (inside_mask[:, 1:].sum(1) > 1)  # overlapping box points
        
    considered_probs = ((db_clusters[:, None] * inside_mask[:,1:]) > 0) * mask_without_background
    
    non_overlaps = (inside_mask[:, 1:].sum(1) == 1) # non-overlapping box points
    non_over_probs = (considered_probs * non_overlaps[:, None])[:1, non_overlaps]
    non_over_probs = non_over_probs[non_over_probs > 0]
    
    overlap_probs = (considered_probs * overlaps[:, None])[:1, overlaps]
    overlap_probs = overlap_probs[overlap_probs > 0]
      
    loss = - torch.log(non_over_probs).mean()# + torch.log(overlap_probs).mean() 
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# inside_mask.shape, logs_mask[:1].shape 

In [ ]:
# plt = visualize_boxes_and_masks(pc, boxes, db_clusters)

plt = visualize_boxes_and_masks(pc, boxes, len(boxes) * (inside_mask[:,8]).detach().cpu())
imshow(plt)
# visualize_points3D(pc[0, :, :3].detach().cpu().numpy(), db_clusters.detach().cpu().numpy(), bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)
# visualize_points3D(pc[0, :, :3].detach().cpu().numpy(), inside_mask[8].detach().cpu().numpy(), bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)
# visualize_points3D(pc[0, :, :3].detach().cpu().numpy(), inside_mask[:,1:].sum(1).detach().cpu().numpy(), bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)
visualize_points3D(pc[0, :, :3].detach().cpu().numpy(), logs_mask.argmax(dim=-1)[0].detach().cpu().numpy(), bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)

In [ ]:
for i in tqdm(range(1500)):
    optimizer.zero_grad()
    mask = logs_mask.softmax(dim=-1)
    
    dist, NN, _ = knn_points(pc[:-1, :, :3] + gt_flow[:-1, :, :3], pc[1:, :, :3], lengths1=None, lengths2=None, K=1, return_nn=True)
    
    
    loss = 0
    
    # density-based regularizer
    mean_probs = scatter(logs_mask[:1, db_clusters >=0].softmax(dim=-1), db_clusters[db_clusters >= 0], reduce='sum', dim=1)    # calculate mean mask over clusters
    probs = mean_probs.max(dim=-1)[0] # select the highest probability per instance from masks
    loss += - probs.mean()
    
    # if i > 1000:
    # todo weights on truncated NN distance
    matched_masks = torch.gather(mask[1:], 1, NN[:,:,0].unsqueeze(-1).repeat(1,1,mask.shape[-1]))   # todo unchecked! slightly checked
    continuity_loss = (mask[:-1] - matched_masks).abs()
    
    loss += continuity_loss.mean()
    
    for i in range(len(pc1) - 1):
        smooth_loss, _ = SmoothLosses[i].smoothness_loss(mask[i:i+1], SmoothLosses[i].NN_pc1)
        loss += smooth_loss
        
    loss.mean().backward()  
    optimizer.step()

instances = logs_mask[:-1].argmax(dim=-1).view(-1).cpu().numpy()
visualize_points3D(pc[:-1, :, :3].view(-1,3).detach().cpu().numpy(), instances, bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)
# visualize_points3D(pc[0, :, :3].view(-1,3).detach().cpu().numpy(), logs_mask[0].argmax(dim=-1), bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)

# Use Temporality for refinement - Shape fitting

In [ ]:
# fit one object along time
import torch
from pytorch3d.ops.knn import knn_points
from pytorch3d.transforms import axis_angle_to_matrix
import matplotlib.pyplot as plt
import os

plt.close()
device = torch.device('cuda:0')

# for i in range(len(centers)):
    # centers[i] += i
frames = 3
pts = torch.stack([torch.rand(30, 3) * 1 + torch.tensor((i * 1.5, 3 * torch.tensor(i * 0.3).sin(), i)) for i in range(frames)]).to(device)

centers = torch.stack([pts[i][1] for i in range(len(pts))]).to(device) # random point init

# initialize yaw from initialized centers
x = torch.diff(centers[:,0])# append=torch.diff(centers[-1:,0]))
y = torch.diff(centers[:,1])# append=torch.diff(centers[-1:,1]))
angles = torch.atan2(y, x)
yaw = torch.cat((angles, angles[-1:]), dim=0)[:,None].to(device) # append second last as a approximation of last one
yaw.requires_grad_(True)



# centers = torch.ones((5,3))    
centers.requires_grad_(True)

# next add rotation
optimizer = torch.optim.Adam([centers, yaw], lr=0.01)

    
# batch size = time
# for each cluster, match 1-NN
# gather all combinations? ne, to musi byt zaroven, nektery body proste nemusi mit match

# todo add coverage
# todo freespace in matching the object. Just generate few points in direction to sensor and do not let any point be near it
# todo match only in consecutive times
# normals in registration?
for e in range(85):
    fig, ax = plt.subplots(1,2)
    loss = 0
    
    # calculate 
    rotations = torch.cat((torch.zeros(len(pts),2, device=device), yaw), dim=1)
    rot_mats = axis_angle_to_matrix(rotations)
    rot_mats = rot_mats.permute(0,2,1) # compensate transposition
    transformed_pts = torch.bmm(pts - centers[:, None, :] - centers[:1, None, :], rot_mats) # shift to local object origin center and also put global origin to first center
        
    # Iterate all with all
    for i in range(len(centers)):
        except_mask = torch.arange(len(pts)) != i    
        
        dist, NN, _ = knn_points(transformed_pts[i:i+1], transformed_pts[except_mask].view(1,-1,3), K=1)
        back_dist, NN, _ = knn_points((transformed_pts[except_mask]).view(1,-1,3),transformed_pts[i:i+1], K=1)
        
        # dist, NN, _ = knn_points(pts[i:i+1] - centers[i:i+1] - centers[0], (pts[except_mask] - centers[except_mask, None] - centers[0]).view(1,-1,3), K=1)
        # back_dist, NN, _ = knn_points((pts[except_mask] - centers[except_mask, None] - centers[0]).view(1,-1,3), pts[i:i+1] - centers[i:i+1] - centers[0], K=1)
        loss += dist.mean() + back_dist.mean()
    
    smooth_traj = (torch.diff(centers, dim=0) ** 2).mean() + (torch.diff(rotations, dim=0) ** 2).mean()
    
    loss += smooth_traj
    
    loss.mean().backward()
    optimizer.step()
    optimizer.zero_grad()
        
    for i in range(len(pts)):
        p = pts[i].detach().cpu().numpy()
        ax[0].scatter(p[:,0] - centers[i, 0].detach().cpu().numpy(), p[:,1] - centers[i, 1].detach().cpu().numpy())
        
    for i in range(len(pts)):
        p = pts[i].detach().cpu().numpy()
        ax[1].scatter(p[:,0], p[:,1])
        ax[1].scatter(centers[i, 0].detach().cpu().numpy(), centers[i, 1].detach().cpu().numpy(), c='k', s=100)
        
        
        vis_arrow = torch.stack((centers[i, :2], centers[i, :2]))
        
        vis_arrow[1] += torch.tensor((1, torch.tan(yaw)[i]), device=device)
        ax[1].plot(vis_arrow[:,0].detach().cpu().numpy(), vis_arrow[:,1].detach().cpu().numpy(), 'g-')
        
    ax[0].axis('equal')
    # ax[1].axis('equal')
    ax[1].set_ylim()
    fig.savefig(f'imgs/{e:03d}.png', facecolor='white', transparent=False)
    plt.close()


        
print(centers, yaw)



In [ ]:
# calculate 


In [ ]:

cluster_mask = (instances == 2)
# times
cluster_pc = pc[:-1].view(-1,4)[cluster_mask]

vis_pc = cluster_pc.reshape(-1,4).detach().cpu().numpy()

# initialize centers and rotations (flow based?)
from torch_scatter import scatter
from pytorch3d.transforms import axis_angle_to_matrix
centers = scatter(cluster_pc[:, :3], cluster_pc[:, 3].long(), dim=0, reduce='mean') # todo later shift for geometrical mean?
rotations = torch.eye(3).repeat(len(centers), 1, 1).to(device)

# linear smoothness
dt = 1
d_xyza = torch.diff(centers, dim=0) / dt
print(d_xyza)

# axis_angle_to_matrix()

# shift to center and rotate for shape fitting, find the rotations and centers that minimize the shape displacements


# visualize_multiple_pcls(*[vis_pc[:,:3], centers.detach().cpu().numpy()])

# visualize_points3D(vis_pc[:,:3], vis_pc[:,3], bg_color=(0.5,0.5,0.5,0), show_grid=False, show_axis=False)
# visualize_flow_and_instances(pc1[0].detach().cpu(), flow=None, mask=mask[0].detach().cpu())
# visualize_flow_and_instances(pc1[1].detach().cpu(), flow=None, mask=mask[1].detach().cpu())
# NN[0,:,0].shape, NN.max(), mask.shape

In [3]:

import os
os.system('CUDA_LAUNCH_BLOCKING=1')
from vis.deprecated_vis import *
from data.dataloader import NSF_dataset, SFDataset4D
import torch

device = torch.device('cuda:0')
dataset = NSF_dataset(dataset_type='valeo_filip')

# 
first = 0
max_radius = 70

for file in dataset.all_files[first:first+1]:
    data = np.load(file, allow_pickle=True)

    pc1 = data['pc1']
    pc2 = data['pc2']
    
    pc1_mask = np.linalg.norm(pc1, axis=-1) < max_radius
    pc1 = pc1[pc1_mask]
    pc2 = pc2[np.linalg.norm(pc2, axis=-1) < max_radius]
    pc1 = pc1[pc1[:, 2] > 0.3]  # remove ground
    pc2 = pc2[pc2[:, 2] > 0.3]  # remove ground


pc1 = torch.from_numpy(pc1[None,:]).to(device)
pc2 = torch.from_numpy(pc2[None,:]).to(device)

pose0 = np.load(dataset.all_files[first-1], allow_pickle=True)['pose']
pose1 = data['pose']    

# curr_pose = np.linalg.inv(pose1) @ pose0
# curr_pose = torch.from_numpy(curr_pose).to(device).to(torch.float32)

# shifted_pc1 = pc1[0] @ curr_pose[:3,:3].T + curr_pose[:3,3]

# pc1 = shifted_pc1.unsqueeze(0)
# visualize_multiple_pcls(*[pc1[0].detach().cpu().numpy(), shifted_pc1.detach().cpu().numpy(), pc2[0].detach().cpu().numpy()])


In [3]:
import pandas as pd
from vis.deprecated_vis import *
from tqdm import tqdm
from loss.flow import GeneralLoss, SC2_KNN
from models.NP import NeuralPriorNetwork, PoseNeuralPrior


from pytorch3d.ops.knn import knn_points
knn_points(pc1, pc1)

tmp_cfg = {
           'lr': [0.001],
           'K': [8],
           'beta': [0.95],
           'max_radius': [2],
           'grid_factor': [10],
           'smooth_weight': [0],
           'forward_weight': [0],
           'sm_normals_K': [0],
           'init_cluster': [True],
           'early_patience': [100],
           'max_iters': [150],
           'use_normals' : [False],    # 0, 1 - use normals for SC2 KNN search
           'init_transform' : [0],    # 0 - init as eye matrix, 1 - fit transform by NN to pc2 as init
           'use_transform' : [1],     # 0 - do not use trans, 1 - sum rigid and pred flow, 2 - transform is input to model
           'flow_output' : ['flow'],  # flow, rigid
           'SC2' : ['SC2_KNN'],          # MBSC, SC2_KNN
            }

cfg = tmp_cfg
cfg = pd.DataFrame(tmp_cfg).iloc[0].to_dict()
print(cfg)

model = PoseNeuralPrior(pc1, pc2, init_transform=cfg['init_transform'], use_transform=cfg['use_transform']).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=cfg['lr'])

LossModule = GeneralLoss(pc1=pc1, pc2=pc2, dist_mode='knn_points', K=cfg['K'], max_radius=cfg['max_radius'],
                         smooth_weight=cfg['smooth_weight'],
                         forward_weight=0, sm_normals_K=cfg['sm_normals_K'], pc2_smooth=True)

st = time.time()

SC2_Loss = SC2_KNN(pc1=pc1, K=cfg['K'], use_normals=cfg['use_normals'])

for flow_e in tqdm(range(cfg['max_iters'])):
    
    pc1 = pc1.contiguous()

    pred_flow = model(pc1)

    loss = LossModule(pc1, pred_flow, pc2)
    
    loss += cfg['beta'] * SC2_Loss(pred_flow)


    loss.mean().backward()

    optimizer.step()
    optimizer.zero_grad()
    
eval_time = time.time() - st


FastGeodis not found, will not use it. This is not error, just future work
Cupy works only on GPU or is not found, will not use it. This is not error, just future work
{'lr': 0.001, 'K': 8, 'beta': 0.95, 'max_radius': 2, 'grid_factor': 10, 'smooth_weight': 0, 'forward_weight': 0, 'sm_normals_K': 0, 'init_cluster': True, 'early_patience': 100, 'max_iters': 150, 'use_normals': False, 'init_transform': 0, 'use_transform': 1, 'flow_output': 'flow', 'SC2': 'SC2_KNN'}


100%|█████████████████████████████████████████████████████████████████| 150/150 [00:52<00:00,  2.83it/s]


In [9]:
# data['pose']

# todo pridat intuitivni popis na cem je zalozena supervize a jak to souvisi s parametrama
# todo priklad postupu, kdybych to chtel spustit na novych datech (Pandar), jak bych to pouzil
# todo ukaz i na waymo, kde to funguje dobre

# Kabsch
from ops.transform import find_weighted_rigid_alignment
weights = torch.ones((1,pc1.shape[1])).to(device)

trans = find_weighted_rigid_alignment(pc1, pc1 + pred_flow, weights)[0]

kabsch_shifted_pc1 = (pc1[0] @ trans[:3,:3].T + trans[:3,3]).unsqueeze(0)
visualize_multiple_pcls(*[pc1[0].detach().cpu().numpy(), kabsch_shifted_pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy()])
time_label = torch.cat((torch.zeros(pc1.shape[1]), torch.ones(pc2.shape[1]))).to(device)

# cat_pc = torch.cat((kabsch_shifted_pc1.unsqueeze(0), pc2), dim=1)
# visualize_points3D(cat_pc[0].detach().cpu().numpy(), time_label.detach().cpu().numpy(), bg_color=(1,1,1,0), show_grid=False, show_axis=False)

# Dynamic motion
compensated_flow = kabsch_shifted_pc1 - pc1
object_flow = pred_flow - compensated_flow
dynamic_mask = object_flow.norm(dim=-1) > 0.15


visualize_points3D(pc1[0], dynamic_mask[0], bg_color=(1,1,1,0), show_grid=False, show_axis=False)

# visualize_points3D(pc1[0], object_flow[0].norm(dim=-1), bg_color=(1,1,1,0), show_grid=False, show_axis=False)

# visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0,:1].detach().cpu().numpy(), object_flow[0].detach().cpu().numpy())
# visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy(), object_flow[0].detach().cpu().numpy())
# visualize_flow3d(pc1[0].detach().cpu().numpy(), pc2[0].detach().cpu().numpy(), pred_flow[0].detach().cpu().numpy())

/mnt/appl/software/SciPy-bundle/2021.05-foss-2021a/lib/python3.9/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [7]:
from sklearn.cluster import DBSCAN
clusters = DBSCAN(eps=0.5, min_samples=5).fit_predict((pc1+pred_flow)[0].detach().cpu().numpy()) + 1
clusters[dynamic_mask[0].cpu() == 0] = 0

# order instances as integers
for i, idx in enumerate(np.unique(clusters)):
    if idx == 0: continue
    clusters[clusters == idx] = i
    
visualize_points3D(pc1[0].detach().cpu().numpy(), clusters, bg_color=(1,1,1,0), show_grid=False, show_axis=False)

In [4]:
# Cosine sim
# same ---> 1
# orthogonal ---> 0 
# opposite ---> -1
import torch.nn.functional as F
from loss.sc_utils import power_iteration, spatial_consistency_score
feats = torch.randn(2, 4, 2, device=device, requires_grad=True)    # Number of neighborhoods, points in one neighborhood, features

def construct_cosine_sim_matrix(features):
    ''' Features:  Number of neighborhoods, points in one neighborhood, features '''

    cosine_sim_matrix = F.cosine_similarity(features[..., None, :, :], features[..., :, None, :], dim=-1)

    return cosine_sim_matrix

def calculate_cosine_sim_loss(cosine_sim_mat):

    lead_eigvec = power_iteration(cosine_sim_mat)
    
    sc_score = spatial_consistency_score(cosine_sim_mat, lead_eigvec)
    
    sc_loss = - torch.log(sc_score)

    return sc_loss

optimizer = torch.optim.Adam([feats], lr=0.01)


for e in range(200):
    
    cosine_sim_mat = construct_cosine_sim_matrix(feats)
    sc_loss = calculate_cosine_sim_loss(cosine_sim_mat)

    
    loss = sc_loss.mean()

    # Another version of pushing features together without outliers
    # cosine_loss = (1 - cosine_sim_mat).norm()
    # feature_norm_loss = (1 - feats.norm(dim=-1)).norm()
    # loss = cosine_loss + feature_norm_loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

print(loss)
print('Final Feats: ', feats, cosine_sim_mat, sc_score)

tensor(0.0003, device='cuda:0', grad_fn=<MeanBackward0>)


NameError: name 'sc_score' is not defined

# Notes
- minimizing (1 - cosine_sim_mat) leads to make them all similar, works with norm ||1|| too on some runs
- minimizing sc_loss leads group similar and push away outliers in cosine_feature space

In [6]:
# Point cloud
from vis.deprecated_vis import * 
from pytorch3d.ops.knn import knn_points
_, nn, _ = knn_points(pc1, pc1, K=16)

nbr_feats = 3
features = torch.randn(pc1.shape[0], pc1.shape[1], nbr_feats).to(device) * 0.3
features = features.requires_grad_(True)

neigh_feats = features[0, nn[0]]

optimizer = torch.optim.Adam([features], lr=0.01)

visualize_flow3d(pc1[0], pc1[0, 1:], features)

for e in range(500):
    
    neigh_feats = features[0, nn[0]]

    cosine_sim_mat = construct_cosine_sim_matrix(neigh_feats)

    # Regularization with outliers
    sc_loss = calculate_cosine_sim_loss(cosine_sim_mat)
    loss = sc_loss.mean()

    # cosine_loss = (1 - cosine_sim_mat).norm()
    # feature_norm_loss = (1 - feats.norm(dim=-1)).norm()
    # loss = cosine_loss + feature_norm_loss
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()

    print(loss)

visualize_flow3d(pc1[0], pc1[0, :1], features)

tensor(0.7685, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.7284, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6600, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6289, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5999, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5727, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5475, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5239, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5019, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4813, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4621, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4441, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4272, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.4113, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3964, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3824, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3691, device='cuda:0',